# Beta-Bernoulli Classification with NGBoost

**Problem**: Binary classification with calibrated uncertainty. Unlike standard
classifiers that output a single probability, the Beta-Bernoulli gives you a
full *distribution over the probability itself* via its Beta(alpha, beta) prior.

**When to use**:
- Medical diagnosis: not just "80% chance of disease" but "somewhere between 60-95%"
- Fraud detection: flag transactions where the model is genuinely uncertain
- A/B testing: quantify uncertainty in conversion rate predictions
- Any binary outcome where knowing *how confident the model is* matters

**How it works**: The predictive probability for class 1 is `alpha / (alpha + beta)`.
When alpha and beta are both large, the model is confident. When they're small,
there's high uncertainty.

This notebook shows the **classification** pattern using `make_sympy_log_score`
(the lower-level factory) with a thin `ClassificationDistn` wrapper.

In [1]:
import sys
sys.path.insert(0, '..')  # add project root so 'import ngboost' finds the package

In [2]:
import numpy as np
import sympy as sp
import sympy.stats as symstats
from scipy.special import digamma
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from ngboost import NGBClassifier
from ngboost.distns.distn import ClassificationDistn
from ngboost.distns.sympy_utils import make_sympy_log_score

## 1. Define the score class

The BetaBernoulli uses a Beta(alpha, beta) prior, so the predictive probability
is `p = alpha / (alpha + beta)`. The likelihood is Bernoulli(p), and the factory
auto-derives score, gradient, and Fisher Information from that.

In [3]:
alpha, beta, y = sp.symbols("alpha beta y")
p = alpha / (alpha + beta)

BetaBernoulliLogScore = make_sympy_log_score(
    params=[(alpha, True), (beta, True)],
    y=y,
    sympy_dist=symstats.Bernoulli("Y", p),
    name="BetaBernoulliLogScore",
)

print(f"Score class: {BetaBernoulliLogScore}")
print(f"  Has analytical metric: {'metric' in BetaBernoulliLogScore.__dict__}")

Score class: <class 'ngboost.distns.sympy_utils.BetaBernoulliLogScore'>
  Has analytical metric: True


## 2. Build the classification distribution

For classification, NGBoost requires a `ClassificationDistn` subclass with a
`class_probs()` method. This is a thin wrapper — the heavy lifting (score,
gradient, FI) is handled by the factory-generated score class above.

In [4]:
class BetaBernoulli(ClassificationDistn):

    n_params = 2
    scores = [BetaBernoulliLogScore]

    def __init__(self, params):
        super().__init__(params)
        self.alpha = np.exp(params[0])
        self.beta = np.exp(params[1])

    def class_probs(self):
        p = self.alpha / (self.alpha + self.beta)
        return np.column_stack([1 - p, p])

    def fit(Y):
        p = np.clip(np.mean(Y), 0.01, 0.99)
        a, b = p * 2, (1 - p) * 2
        for _ in range(100):
            ab = a + b
            psi_ab = digamma(ab)
            a = np.clip(a * (np.mean(digamma(Y + a)) - psi_ab) / (digamma(a) - psi_ab + 1e-10), 1e-4, 1e4)
            b = np.clip(b * (np.mean(digamma(1 - Y + b)) - psi_ab) / (digamma(b) - psi_ab + 1e-10), 1e-4, 1e4)
        return np.array([np.log(a), np.log(b)])

    def sample(self, m):
        p = np.squeeze(self.alpha / (self.alpha + self.beta))
        return np.random.binomial(1, p, size=m).astype(float)

    @property
    def params(self):
        return {"alpha": self.alpha, "beta": self.beta}


print(f"Distribution: {BetaBernoulli}")
print(f"  Base: {BetaBernoulli.__bases__}")

Distribution: <class '__main__.BetaBernoulli'>
  Base: (<class 'ngboost.distns.distn.ClassificationDistn'>,)


## 3. Load data

We use the breast cancer dataset — a standard binary classification benchmark.

In [5]:
X, Y = load_breast_cancer(return_X_y=True)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
Y_train = Y_train.astype(float)
Y_test = Y_test.astype(float)

print(f"Train: {len(Y_train)} ({Y_train.mean():.1%} positive)")
print(f"Test:  {len(Y_test)} ({Y_test.mean():.1%} positive)")

Train: 455 (62.9% positive)
Test:  114 (62.3% positive)


## 4. Train NGBClassifier

In [6]:
from sklearn.tree import DecisionTreeRegressor
tree_learner = DecisionTreeRegressor(
    criterion="friedman_mse",
    min_samples_split=10,
    min_samples_leaf=3,
    min_weight_fraction_leaf=0.0,
    max_depth=6,
    splitter="best",
    random_state=None,
)
ngb = NGBClassifier(
    Dist=BetaBernoulli,
    n_estimators=1000,
    learning_rate=0.05,
    verbose=False,
    random_state=42,
    Base=tree_learner
)
ngb.fit(X_train, Y_train)
print("Training complete.")

C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:8: RuntimeWarning: overflow encountered in exp
  self.alpha = np.exp(params[0])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:9: RuntimeWarning: overflow encountered in exp
  self.beta = np.exp(params[1])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:8: RuntimeWarning: overflow encountered in exp
  self.alpha = np.exp(params[0])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:9: RuntimeWarning: overflow encountered in exp
  self.beta = np.exp(params[1])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:8: RuntimeWarning: overflow encountered in exp
  self.alpha = np.exp(params[0])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:9: RuntimeWarning: overflow encountered in exp
  self.beta = np.exp(params[1])
C:\Users\guyko\AppData\Local\Temp\ipykernel_15172\1805550954.py:8: RuntimeWarning: overflow encountered in exp
  self.alpha = np.exp(params

ValueError: Input y contains NaN.

## 5. Predictions with uncertainty

The key advantage: not just a probability, but a *distribution* over probabilities.

In [ ]:
# Class predictions
Y_pred = ngb.predict(X_test)
accuracy = np.mean(Y_pred == Y_test)
print(f"Accuracy: {accuracy:.1%}")

# Predicted probability distributions
dists = ngb.pred_dist(X_test)
probs = dists.class_probs()  # (n_samples, 2) — [P(0), P(1)]

print(f"\nFirst 5 test samples:")
print(f"  alpha:  {dists.alpha[:5].round(3)}")
print(f"  beta:   {dists.beta[:5].round(3)}")
print(f"  P(y=1): {probs[:5, 1].round(4)}")
print(f"  Actual: {Y_test[:5].astype(int)}")

## 6. Quantify prediction uncertainty

The Beta distribution over `p` lets us compute confidence intervals.
When alpha and beta are large, the model is confident. When they're
small, there's high uncertainty about the true probability.

In [ ]:
import scipy.stats

# Compute 90% credible intervals for P(y=1)
p_lower = scipy.stats.beta.ppf(0.05, dists.alpha, dists.beta)
p_upper = scipy.stats.beta.ppf(0.95, dists.alpha, dists.beta)
p_mean = dists.alpha / (dists.alpha + dists.beta)

# Width of credible interval = uncertainty
uncertainty = p_upper - p_lower

print("90% credible intervals for P(y=1) — first 10 test samples:")
print(f"{'idx':>4s}  {'P(y=1)':>8s}  {'90% CI':>18s}  {'width':>8s}  {'true':>5s}")
for i in range(10):
    print(f"{i:4d}  {p_mean[i]:8.4f}  [{p_lower[i]:.4f}, {p_upper[i]:.4f}]  {uncertainty[i]:8.4f}  {int(Y_test[i]):5d}")

print(f"\nMean uncertainty (CI width): {uncertainty.mean():.4f}")
print(f"Most uncertain sample:  idx={np.argmax(uncertainty)}, width={uncertainty.max():.4f}")
print(f"Most confident sample:  idx={np.argmin(uncertainty)}, width={uncertainty.min():.4f}")

## 7. Visualize

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: predicted probabilities with credible intervals, sorted by predicted P(y=1)
ax = axes[0]
order = np.argsort(p_mean)
x = np.arange(len(order))
ax.fill_between(x, p_lower[order], p_upper[order], alpha=0.3, label="90% CI")
ax.plot(x, p_mean[order], "b-", linewidth=0.8, label="P(y=1)")
ax.scatter(x, Y_test[order], c="red", s=8, alpha=0.6, zorder=5, label="true label")
ax.set_xlabel("Test samples (sorted by P(y=1))")
ax.set_ylabel("Probability")
ax.set_title("Predictions with 90% credible intervals")
ax.legend()

# Right: Beta distributions for a confident vs uncertain prediction
ax = axes[1]
p_grid = np.linspace(0.001, 0.999, 300)

# Most confident
i_conf = np.argmin(uncertainty)
pdf_conf = scipy.stats.beta.pdf(p_grid, dists.alpha[i_conf], dists.beta[i_conf])
ax.plot(p_grid, pdf_conf,
        label=f"confident (idx={i_conf}, true={int(Y_test[i_conf])})")

# Most uncertain
i_unc = np.argmax(uncertainty)
pdf_unc = scipy.stats.beta.pdf(p_grid, dists.alpha[i_unc], dists.beta[i_unc])
ax.plot(p_grid, pdf_unc,
        label=f"uncertain (idx={i_unc}, true={int(Y_test[i_unc])})")

ax.set_xlabel("P(y=1)")
ax.set_ylabel("density")
ax.set_title("Beta prior over probability")
ax.legend()

plt.tight_layout()
plt.show()

## Note: classification vs regression

For **classification**, `make_distribution` doesn't apply directly (it creates
`RegressionDistn`). Instead, use the lower-level `make_sympy_log_score` to generate
the score class, then write a thin `ClassificationDistn` wrapper with `class_probs()`.

The score class still handles all the calculus automatically — you only need to
write `__init__`, `class_probs`, `fit`, and `sample` (all straightforward).